# Setting

In [1]:
import sys, os
import pandas as pd

sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")

from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as iC
from Clust.clust.ingestion.mongo import mongo_client
from Clust.clust.ML.common import ML_api
from Clust.clust.ML.tool import meta

influxdb_client = iC.InfluxClient(ins.CLUSTDataServer2)
mongodb_client = mongo_client.MongoClient(ins.CLUSTMetaInfo2)

/Users/kim-youngkee/Documents/CLUSTER/.clust/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cpu is available.


# 1. set param from Front End

In [2]:
# feature list와 data_y_flag는 model_info에서 받아와야 합니다.
# regression
params1 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'regression_energy_cleanLevel4_testX',
        "feature_list": "None"
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'regression_energy_cleanLevel4_testy',
        "feature_list": "None"
    },
    'data_y_flag' : "None",
    'model_name':'regression_energy_cleanLevel4_trainX_regression_LSTM_rg_'
}

# forecasting
params2 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_testX',
        "feature_list": "None"
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_testX',
        "feature_list": "None"
    },
    'data_y_flag' : "None",
    'model_name':'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainX_regression_LSTM_rg_'
}

# classification
params3 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'classification_actionPattern_cleanLevel0_testX',
        "feature_list": "None"
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'classification_actionPattern_cleanLevel0_testy',
        "feature_list": "None"
    },
    'data_y_flag' : "None",
    'model_name':'classification_actionPattern_cleanLevel0_trainX_classification_LSTM_cf_'
}

# 2. Backend

In [3]:
# 1. Backend Parameter Setting
params = params1

# chage tpye string to bool -> ex) 'true' -> True
params = ML_api.chagne_type_str_to_bool(params)

In [4]:
# 2. get model meta
model_name = params['model_name']
meta_file_name = "./meta.json"
# for testing mongodb model
# model_meta = meta.read_model_meta_from_mongodb(mongodb_client, 'model', 'meta', model_name)
# for testing local model
model_meta = meta.read_model_meta_from_local(meta_file_name, model_name)

# data parameters from model meta
params['ingestion_param_X']['feature_list'] = model_meta['ingestion_param_X']['feature_list']
params['ingestion_param_y']['feature_list'] = model_meta['ingestion_param_y']['feature_list']
params['data_y_flag'] = model_meta['data_y_flag']

# scaler/transform info for test data preparation
params['scaler_info'] = model_meta['scaler_param']
params['transform_info'] = model_meta['transform_param']

# model info for ML test
params['model_info'] = model_meta['model_info']

In [5]:
# 3. Data Preparation
# make test data, scaler
test_X_array, test_y_array, scaler_X, scaler_y = ML_api.test_data_preparation(params, influxdb_client)

{'bucket_name': 'integration', 'ms_name': 'regression_energy_cleanLevel4_testX', 'feature_list': ['Press_mm_hg', 'RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7', 'RH_8', 'RH_9', 'RH_out', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed']}
window_size: 144 nan_limit_num: 72
(6048, 24) (42, 144, 24)
(42, 1) (42, 1)


In [6]:
# set scaler by data y flag
if params['data_y_flag']:
    scaler = scaler_y
else:
    scaler = scaler_X

In [7]:
# model test
result = ML_api.ML_test(params, test_X_array, test_y_array, scaler)

features shape: torch.Size([42, 144, 24]) targets shape:  torch.Size([42, 1])

Start testing data

scale
=========================data
        value  prediction
0   17.370001   12.691754
1   20.650000   12.621255
2   11.420000   21.575741
3   10.680000    9.304165
4   12.439999   10.378374
5   11.170000    9.690221
6   24.120001   16.030220
7   10.990000   13.233480
8   13.760000   11.656842
9   14.560000   12.547544
10  14.970000   15.658759
11  13.429999   12.303468
12  11.570000   11.908084
13   9.330000   10.090723
14  15.580000   14.225930
15  20.929998   16.864517
16  11.990000   10.612164
17  15.370000   11.329812
18  10.470000   11.480734
19  14.410000   12.548252
20  10.160000   10.276198
21  15.120000   19.070885
22  12.320000   10.652502
23  10.460000   12.146081
24  15.060000    7.756890
25   9.680000   14.774932
26  10.690000   11.165871
27  17.799997   22.977623
28  10.690000    9.301559
29  17.059998   10.612042
30  20.879999   20.170544
31  10.890000    9.863614


In [8]:
result

{'result': {'index': [0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31],
  'value': {'value': [17.3700008392334,
    20.649999618530273,
    11.420000076293945,
    10.680000305175781,
    12.439998626708984,
    11.170000076293945,
    24.1200008392334,
    10.989999771118164,
    13.760000228881836,
    14.5600004196167,
    14.970000267028809,
    13.429999351501465,
    11.569999694824219,
    9.329999923706055,
    15.579999923706055,
    20.92999839782715,
    11.989999771118164,
    15.369999885559082,
    10.470000267028809,
    14.40999984741211,
    10.15999984741211,
    15.119999885559082,
    12.319999694824219,
    10.460000038146973,
    15.0600004196167,
    9.680000305175781,
    10.6899995803833,
    17.799997329711914,
    10.6899995803833,
    17.05999755859375,
    20.8799991607666,
    10.890000343322754],

In [9]:
model_meta['model_info']

{'model_purpose': 'regression',
 'model_method': 'LSTM_rg',
 'model_name': 'regression_energy_cleanLevel4_trainX_regression_LSTM_rg_',
 'model_tags': 'tagstest',
 'train_parameter': {'lr': 0.0001,
  'weight_decay': 1e-06,
  'n_epochs': 1000,
  'batch_size': 16,
  'device': 'cpu'},
 'model_parameter': {'rnn_type': 'lstm',
  'input_size': 24,
  'hidden_size': 64,
  'num_layers': 2,
  'output_dim': 1,
  'dropout': 0.1,
  'bidirectional': True},
 'seq_len': 144,
 'input_size': 24,
 'model_file_path': {'modelFile': {'fileName': 'model.pth',
   'filePath': './Models/LSTM_rg/regression_energy_cleanLevel4_trainX_regression_LSTM_rg_/regression_energy_cleanLevel4_trainX/model.pkl'}}}